In [1]:
import os, time
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
from scipy.linalg import expm, eig, logm
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, BasicAer, execute, IBMQ
from operators.triangle_plaquette_hamiltonian import TrianglePlaquetteHamiltonian
from operators.pauli_hamiltonian import PauliHamiltonian
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
from qiskit.providers.jobstatus import JobStatus
from qiskit.visualization import plot_histogram


In [2]:
# Get on one of the IBM Q machines
IBMQ.load_account() # Load account from disk
print(IBMQ.providers())    # List all available providers
provider = IBMQ.get_provider(hub='ibm-q-ornl', project='brookhaven')
provider
provider.backends()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>, <AccountProvider for IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>]


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>,
 <IBMQBackend('ibmq_johannesburg') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brook

In [3]:
backend = provider.get_backend('ibmq_cambridge')
backend

<IBMQBackend('ibmq_cambridge') from IBMQ(hub='ibm-q-ornl', group='hep-qis', project='brookhaven')>

In [ ]:
# 2 triangle plaquettes
#backend = BasicAer.get_backend('qasm_simulator')
T = 0.1
H = TrianglePlaquetteHamiltonian(1, 1, 2)
# Construct Circuit with n_steps = 20
qr = QuantumRegister(6, 'qr')
cr = ClassicalRegister(6, 'cr')
circ1 = QuantumCircuit(qr, cr)
circ1 = H.trotter_circuit_optimized(circ1, qr, T, 1)
circ1.measure(qr, cr)
job = execute(circ1, backend, shots=1024)
lapse = 0
interval = 30
while job.status() != JobStatus.DONE:
    print('Status @ {} seconds'.format(interval * lapse))
    print(job.status())
    time.sleep(interval)
    lapse += 1
print(job.status())
result_sim = job.result()
tr_dist2 = result_sim.get_counts(circ1)
plot_histogram(tr_dist2)

Status @ 0 seconds
JobStatus.INITIALIZING
Status @ 30 seconds
JobStatus.QUEUED
Status @ 60 seconds
JobStatus.QUEUED
Status @ 90 seconds
JobStatus.QUEUED
Status @ 120 seconds
JobStatus.QUEUED
Status @ 150 seconds
JobStatus.QUEUED
Status @ 180 seconds
JobStatus.QUEUED
Status @ 210 seconds
JobStatus.QUEUED
Status @ 240 seconds
JobStatus.QUEUED
Status @ 270 seconds
JobStatus.QUEUED
Status @ 300 seconds
JobStatus.QUEUED
Status @ 330 seconds
JobStatus.QUEUED
Status @ 360 seconds
JobStatus.QUEUED
Status @ 390 seconds
JobStatus.QUEUED
Status @ 420 seconds
JobStatus.QUEUED
Status @ 450 seconds
JobStatus.QUEUED
Status @ 480 seconds
JobStatus.QUEUED
Status @ 510 seconds
JobStatus.QUEUED
Status @ 540 seconds
JobStatus.QUEUED
Status @ 570 seconds
JobStatus.QUEUED
Status @ 600 seconds
JobStatus.QUEUED
Status @ 630 seconds
JobStatus.QUEUED
Status @ 660 seconds
JobStatus.QUEUED
Status @ 690 seconds
JobStatus.QUEUED
Status @ 720 seconds
JobStatus.QUEUED
Status @ 750 seconds
JobStatus.QUEUED
Status @ 78

In [ ]:
# Measure distances with the two distribution
def KL(p1, p2):
    """
    Compute the D_KL(p1||p2)
    """
    return np.sum(np.where(p1 != 0, p1 * np.log(p1 / p2), 0))

def H(p):
    """
    Compute Shannon entropy of p
    """
    return -np.sum(np.where(p1 != 0, p1 * np.log(p1), 0))
p1 = np.array([0, 0.5, 0.5, 0]) # Ideally we are expected to observe 50% 01 and 50% 10
p2 = np.array([0.103, 0.498, 0.355, 0.044])
print(KL(p1, p2))
print(H(p1))

In [ ]:
def dec_to_binary_str(dec, n_dig):
    s = bin(dec).replace("0b", "")
    return '0'*(n_dig - len(s)) +s
print([dec_to_binary_str(i, 6) for i in np.where(np.fromiter(tr_dist2.values(), float) > 0.01)[0]])

In [ ]:
plot_histogram(tr_dist2)